In [9]:
import sys
sys.path.insert(0, "../pismemulator/")

from nnemulator import PISMDataset

In [13]:
dataset = PISMDataset(data_dir="../data/speeds_v2/",
                      samples_file="../data/samples/velocity_calibration_samples_100.csv",
                      target_file="../data/observed_speeds/greenland_obs_g1800m.nc",
                     target_corr_threshold=25)
            

The following simulations are missing:
   [337, 595, 539, 542]
  ... adjusting priors
  Loading data sets...


996it [00:28, 35.10it/s]


100000.0


In [1]:
cd ../speedemulator/

/Users/andy/base/pism-emulator/speedemulator


In [8]:
run sample_posterior.py --emulator_dir 2021_11_gris_2km_cpu --data_dir ../data/speeds_v2/ --target_file ../data/observed_speeds/greenland_obs_g1800m.nc

The following simulations are missing:
   [337, 595, 539, 542]
  ... adjusting priors
  Loading data sets...


996it [00:33, 29.50it/s]


ValueError: zero-size array to reduction operation maximum which has no identity